In [12]:
import pandas as pd

train_df = pd.read_csv('A3_task1_data_files/train.csv', sep='\t')
dev_df = pd.read_csv('A3_task1_data_files/dev.csv', sep='\t')

dev_df.rename(columns={'setence1': 'sentence1'}, inplace=True)

# Drop rows with null values
train_df.dropna(inplace=True)
dev_df.dropna(inplace=True)

In [13]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
from transformers import BertTokenizer
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /home/arnav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/arnav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/arnav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/arnav/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
class Task1B_Dataset(Dataset):
    def __init__(self, dataframe, max_length = 128):
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text1 = str(self.dataframe.iloc[idx]['sentence1'])
        text2 = str(self.dataframe.iloc[idx]['sentence2'])
        score = self.dataframe.iloc[idx]['score']
        score = (score / 2.5) - 1
        return text1, text2, score

In [15]:
# Make dataset class
train_dataset = Task1B_Dataset(train_df)
dev_dataset = Task1B_Dataset(dev_df)

# Initialise data loaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=64, shuffle=False)

In [16]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
model = SentenceTransformer("all-MiniLM-L6-v2")

In [19]:
# Function to compute cosine similarities between pairs of sentences
def compute_cosine_similarities(dataset):
    similarities = []
    model.eval()
    with torch.no_grad():
        for sentence1, sentence2, _ in DataLoader(dataset, batch_size=64, shuffle=False):
            sentence1_embeddings = model.encode(sentence1, convert_to_tensor=True)
            sentence2_embeddings = model.encode(sentence2, convert_to_tensor=True)
            cosine_scores = util.pytorch_cos_sim(sentence1_embeddings, sentence2_embeddings)
            cosine_scores_diagonal = cosine_scores.cpu().numpy().diagonal()
            scaled_scores = (cosine_scores_diagonal + 1) * 2.5
            similarities.extend(scaled_scores)

    return np.array(similarities)

In [20]:
train_similarities = compute_cosine_similarities(train_dataset)
dev_similarities = compute_cosine_similarities(dev_dataset)

In [21]:
train_labels = train_df['score'].tolist()
dev_labels = dev_df['score'].tolist()

In [24]:
from scipy.stats import pearsonr
pearson_corr, _ = pearsonr(dev_similarities, dev_labels)
pearson_corr

0.863142382392328